In [1]:
# Copyright(C) 2021 刘珅珅
# Environment: python 3.7
# Date: 2021.2.25
# 字符串替换：lintcode 841

In [35]:
class Solution:
    """
    @param a: The A array
    @param b: The B array
    @param s: The S string
    @return: The answer
    """
    def stringReplace(self, a, b, s):
        # Write your code here
        if not a or not b or not s:
            return s
        
        base = 33
        HASH_SIZE = 100000
        
        ## 计算数组a中所有字符串的hash值
        a_hash = []
        for i in range(len(a)):
            hash_code = 0
            for ch in a[i]:
                hash_code = (ord(ch) + hash_code * base) % HASH_SIZE
            a_hash.append(hash_code)
            
        ## 计算字符串s中所有从头开始的子串的hash
        s_hash = []
        base_powers = []  ## 类似于Robin karp算法中的power
        base_power = 1
        hash_code = 0
        
        ## 这里先分别在s的子串hash数组和power数组中添加一个值
        ## 是为了后续计算不从起始位置开始的子串的hash值方便，添加后，s_hash和base_powers中的索引对应的就是子串的长度了
        s_hash.append(hash_code)
        base_powers.append(base_power)
        for ch in s:
            hash_code = (ord(ch) + hash_code * base) % HASH_SIZE
            s_hash.append(hash_code)
            base_power = (base_power * base) % HASH_SIZE
            base_powers.append(base_power)
        
#         print(a_hash)
#         print(s_hash)
        i = 0
        s_length = len(s)
        ans = ""
        while i < s_length:
            index = -1
            max_length = 0
            
            ## 查找数组a中每个字符串从s中的i位置开始是否匹配后面的字符串，并选择匹配长度最大的替换
            ## 替换后的字符串不再进行替换
            for j in range(len(a)):
                a_str_length = len(a[j])
                if a_str_length + i > s_length:
                    continue
                
                """
                字符串a[j]的hash值已经计算过，现在需要计算字符串s从i开始的长度为a_str_length的子串的hash
                其实就是计算s中不从起始位置开始的子串的hash值，其计算规则是：
                hash(s[i:j]) = (hash(0:j) - hash(0:i) * base^(j-i)) % HASH_SIZE
                例如s = "abcde"，求子串"de"的hash
                先不考虑溢出, hash(s) = (a * base ^ 4 + b * base ^ 3 + c * base ^ 2 + d * base + e) % HASH_SIZE
                hash("abc") = (a * base ^ 2 + b * base + c) % HASH_SIZE
                则(hash(s) - hash("abc") * base ^ (4 - 2)) % HASH_SIZE = (hash(s) - (a * base^4 + b * base^3 + c * base ^ 2) % HASH_SIZE) = (d * base + e) % HASH_SIZE = hash("de")
                """
                ## 例：i = 1, a_str_length = 3，则s_hash[i + a_str_length]=s_hash[4]其实是s[0:4]的hash值
                ## s_hash[i]=s_hash[1]是s[0:1]的hash值，base_powers[a_str_length]=base_powers[3]是base^3
                s_child_hash = (s_hash[i + a_str_length] - s_hash[i] * base_powers[a_str_length]) % HASH_SIZE
                if s_child_hash < 0:
                    s_child_hash += HASH_SIZE
                
                if s_child_hash == a_hash[j] and a_str_length > max_length:
                    ## 这里要判断是否真的是相等的，偶尔会出现hash值相等，但字符串本身不相等的情况
                    for k in range(a_str_length):
                        if a[j][k] != s[i + k]:
                            break
                    else:
                        max_length = a_str_length
                        index = j
            
            ## 在s中的i位置没有发生任何匹配
            if index == -1:
                ## 将i位置的字符添加到ans中
                ans = ans + s[i : i + 1]
                i += 1
            else:
                ans = ans + b[index]
                i += max_length
        return ans
                    
            

In [36]:
solution = Solution()
a = ["ab","aba"]
b = ["cc","ccc"]
s = "ababa"
a = ["...","bacacccaccbccbabaaaacbccbaa","cbbcccbbbbacaaccabbacaaacaaabbbbacbcabcbbbbbbaabaabbaccabbaccbbbabbbabcaacbbcbaabaacaacbbcbbcccabcccbcccbcaacbbcbcaccaabbaabbaccbaabcabacbbaabbcbccccbbbacbbbcacbacaacbbcbcababbabbbaaabbcacbabaaaaaacbabaacaabcccccbacaccbabaccaaacababaccabbbc","cccabccccccaaacaacbcacbccabaabcccccbaaababccacbaaabaccbaccccbcaabbaacaacacabbabbcbbbacbbccbbccbcaabbcbaaababcbbcacacabcaabbaaabbcccacbcaccacbaabbbacccabbcbabbbcaaabacacbabcacabacaccabacbacbcbbaccbbabbbcacbbcccbbcaacbacbbaaabbcbcbbbbcabbcbcabcbcbbcabccacabaabccbcbcccbcaaabacabccccbacccaaccbbbaacbbccbbcbacabbcbabbbacaacccbabccbcccabbacabaaccbbbcbcccbcacbbbbbababbcbb","accbbcaacaacbcacacbaaacbbbcbabbcbaccaababccbbcacabacaabacccabaacabccaccccacbbbcaaacbabaccacbbabcabbacbccaacbcacbbbcbbbbccccaababbbacababbcbbccbbabcaccc","ababbccabaaaacbcbbaccabcaaccbcabcaccaccbababbbbabcaccbbcccaaabbbbaabccccabcabaccbabbabbbcababbccaacbcccbbbaabbbabacaababbabcbbbbccbbccbbcbbbcbcaaabcbcabaaabaccbaccbbbcaacaccbaccaabaaccabcaabbcababaaabccabaaababacbcbbabccbbbbbaccbcccacabbccaaccbccabacbbaabcbccbaacccbbbbaaacbcabcbcaaccababbbabcbcabbccbbcbabbbacaccbcacabacbbbbbbababacccacaacabcabbcabcbbcccacbaabcbabbbaaaabccbababbabcabbccaacaaaccaabcaabcaac","babccabccbbbbbcbbaacabaaaabacbcacacbacccabbabbccabbbcbbabcabbbabbccccaacaaaacbaacbaccaccabccaaacaabaaababacaccaacbccccabcacaacacabbbbaabababcbbcbabcbbcbbacacacaabccbbabcccaaacacaaaccbbccccabbcaabbabacaabaacbaacaabcbabbabbbcacaacaccacabccaabbacccacaaaacbabcbaaacabacacbabbaabaaaaaacbcabaaab","cabbcabacacbaacbbbcaaacabbcacabcbcccbbbabbcabccbcacbbacbacbbaaabcaabbaccccaabbacacbbcabbcababacbcbaabbbbaacaccaaabbcaacaabbababbcccababaabccabcbbcabbaabcbccaabcbababaccbcbacaaabcccbbbacbcaaabbabacbccbbaabbbbcbaaabaabbcbbbbacccccbccababcbcacaaccccbabaccabccccbcbcccccbaabaccbabcacccabbcbcccaccbbabbabbcccacacbcaaabacbbbabaabacbbacccbbacbbaaaacabaabbbbabccbcaacbcbabcbbbbbbacabbbabaabbaacbaaabcaabccbcbbaacccbacaabbabcabababcccbbcabbacbbbbcccbbbbbccbccaabbabbbabcbaacccbbcaacaabaabaaaaccbabcbbccccbabaacabaaaccaaacbbbbbcbbaaabcaaaaaacaaaccccbaacbbababcaccbbcbababacccbbcbabbbcbcbcaaccaababbaacbabcbcaccbccbbbacccaacccaacbacaccabacbacabaacbacacacbacacbabacbbbbabcbacbccbacccbaccabcbcbbcbbcacaacbaccbbcccbabbbccaacbcacbcccbcccacbbcccaababccabbcccccbcbabbaaabbaabbcbcbbabbbcbbababbccbcaabbaaacacbababbcaabaacbccaacacaccccbaccbbbbcbbcbccbbccaacacbbcacbbbacbaaabababbbcbcacbabcbacbababbbbcccccbaaaaabaacccbaaaacccabcaaabacbbbbabcccbccacbbbccaabacabcabcccababbcaaabbabccaabaacccccacbbaabbcbacababbbbbaacabcbbcacbaababbacbbcaaabcacbaaaccbbcccbbbcacabbabcacbacaabccaaacacbaccbaccccbbbbabcccaccccbbbabbbcacaabacaababacaccabcabcbaacccacbccaccacbbbabcbcacccacbbbbbcbaaccbbcaabbbcabccccbaacaabccbacbcbbacbcbcacacacbcacccbccabccaacbcbbabbcacbacbaacacbcccacbbbcaaaacccbabcabbabbacabbccbbcabaacacacbcccbaccbcbaabbcababbaababbbbabbaaacccccbbacabcbbccccacaaccacbaccabccabaaccaababbaacacbcacbaaaaccaabcbaccbcabbbcbccaaabcccabcabacbbccacacccbbcbbabacbabaabbcaacabccbbaacbcbaaabbabaacabbbbaababccbcbcbabacccbaabccbabbbbbcacabccabcacabacccbbabcaabcbccabacccbacbaccbcacbcbccabbaacccbccbbbcaaaacaabbbbcaacccacbcbcababbcbbaccaaabbacbcccccbbabbabbcaabbbcbbccbccbabcccbcbccaacabccbcbbccbcbcbcccccccbccacabaacabacacaabaaaabbccbccacbbcbccbcaabbbaabbacabacaabbabcabcacccbcbcabbababacbcbcbabacaccccbbacaccacbababcbcccbccbaaacbbcacbcbcaabcacbcacaacacabcccaaaacccbbaccacbaaabbcbcccba","bbccabbbacbccbabbbacbbcabbcacbcccbcbbaabcbcacbbccaaabccbabbbaccbacbcbbbcacbcabcaacbacaccabaaaaaaaabbbcaacbbcabacacacbbcaabcbccccabacacbacbcbabbaacbcbcbcbcaaaabaacbbbbacbbbcabcbcaccbbaccbabbcabccbaaabacbaacacccaccbcccbcacbcbcbabaacabcabccaabbbaacbbccabccabcbcabccabcbbacbbacbbbaabbcacbaacbcabcababccbbabcacacbbabcacbcbbaacaaabbbcbcabbbaccacaaaacaaacaaccaaacbbacbbcaccaccacbabcbabaabccbaacccabbbabcabaccccacbcbcbbacbccbabcbbabaacbccbabaccacbbcccbbabaaaaabbcaccbcccacacaacbaabbccabbcccbaaabbbaaabbcabacbbabbcbccbbccbccbacccbbccbaaaaabbbbaaaabbcbbbaacacaabcabcbcbbcccbabbabaabbbaacaaaabaccaacacacccaccbaacbbabacbbcbcbcaaaabbbcbcabccbaaacbcabacbaacaaabababccccbccaaacbcbcbbabcabbabaaababbcabccccbcbabcbbcabbbbbbbacbbbcbbcbcbbcbccccbccbbbcbaacbcababacbaaabcbcbabccabbbcababcbbbaccbbaaabccbccabaacccacbbbcbbbaabbccabbccacbccabcbcaacabbbbabccabbabbcacbbbbaaabbcaccccacabccccbbcccbacbccacbbcbababbbcbccabaabccbaacbbababcabcacbbabcaaacbcbbaccabbbbbaabcbcbacacabbcabbcccabaaaacaabbaaacbbbbbcbbcaaaaccabbbacbcccaabcccbbabcbbcbcccbbbcbcaabbccbbbaacccbccccbaababcbccacaccabcaaabbaabcbbbabaacaccacaacbacabbabbcccccccaccbbaccbbcaccbacacccbbcbacbbbbbaccbcbcbcacaaaaacbcaaacbabcababcacabcabbbaccbcabbcbaaaabbccbcabccbcbcbbabbbcccbacbacbbcccabccbaabbabaccaabcccbbaaccacbcbbbabbaccbcbbabcccbcaacbaacaabbcabccbbcabcbacaccccbaacaaaababaabbccbcbcbbbccaacaccaccbacabbbbcccbabbaaaababbbabbaabbcabccaccaacabaabcacaacbacbccaacabacbabccccabaaabaabcbbcaabbaacbcbcbbbbabbccbabaabaaacaaacacccbbbccbcacabbaacccacbbbcacbacabbccccacacacbcbbbcccbccbbabaaaaccaaaacabaacabacaaaabbabccbababccbccbccbbcabbcbcbccccaaacaccabbabcbbacbbccbaabcbccaabbcbbbabbcabaaaaacaccacbbcccccbbcbabccaacbaabaacccacbabbabbcbcaabbccbcbcccacababcbabcbabbccccbbcaacbcccacaabcbcbbbbcaaccbacbbaacbaaccbcaccaabccaabcbcbababbbcbcbcacccbcacbabcbbcccabcabacbacabcbabcabcbabbbabacabbcacbabcabbcbabbcbabaacbbcbbacbacccbbacaaaaaacbbaabbaab","bbaccbccbbbcabacbcaaaccccbaaabacbaaabcbcabacbaaaaccbacccbaabbaababaccbbacacabaababacbccacbccbbbaababaccccbbcbcbacbcacacabcaaccaaabcbbaaabcbaabbacaacccbbcacbbccabcaaaaacabbcccabaacbccbbbbab","acbbaaaaabbcabbcbacaaabacaaccbcccabccaccaaaabbccbcccccbcbacacbabccaaccbbacabababccacaabbbbbacbcbaabbaaaaabbcbbabccbaccbacbbccbacbabacbbbbbcacbabccaabaacacabcbbbccccbbaacaaaccabcbaabbabcaaababbbacabccbbcabcbbcaacbabccacabccbbcaacccbaccccaacbabbbcacbcbacacccccbccbccacabbbbacaccaaaccbabacbbbbacccaaccabbccbbaccacbbbbcabcabbccbabbbbbbcbcabaabbbcbbccbcbcaacacbcbcbcaccbbcccbaacbaccbcacaaccabaaccbabababbcacaaabcccbcbacbbcbabacaaabbcbbaacaacbaaacbabcccbbbacaacaccbabababbcaaccabaaaccacabcbaccaccaccbbbcbbcacbcabcbcababbbabcbbbccbcabbcabccbaaaacaccccccccbccaaaabcbbcccccbabcababbbcbaabbccccaaaccacacccccccabbaacabbcaaacbbbacaabccbbacbcaaacbbccbbbbbcbbccbcacccaabcacbaaaaaacccbacbcbab","cabbbcaababacaabababcbbabaaaacbcbbccbcbbbbacbacbbaaabacaacccbaabcbccbaccacccaacabbcaaabccbcccacbbacccccacbbabbaaccccbaabaabbccabbbbbccbcccabbabaabbacbcabbcbcabbccbbcaacbaacbaaacbbcbccccbccbcaaacbaaacbacaabcababbbabcacacbbbbabababaacaacbcbcccacbcacaababbbccccbcbbacabbbbbacaaaaccaabbccaacbcbcbbaacbbaccaccbaccbbbcbaabacccabccbabcacaacaacabbbbabcbccbcacbccaabbbbabcbbaccaabaabccbaaabaacccbaabbbcacccbccbcbaacaaabcbccaaaccbccbccbacbcbbccc","acbcccbaaaacabaabcbccbaacabacacacbbbaccbabaabcaabcabbccabccccbcbcacbbbacabcbcacaabbacbacbaaabaacccabcbcccaaababbacbaaababaaaccabbcacccbabcbbbcbaababbbcaaccbabcaacbbccbbcbabaaaaccabcaabcbbbcabcaaccabbabcbccabbbbbcbcbaaaaacbbabaabaaacaabbcaabcbcbacbaaabbabbbccbcccbccbcacccbccbbabaacbaaaaaacbbabcabcaaaccbcaabccababacbbbabcbbbcaccacabcaabcabacccbbccababababccbccbcbaaabacaaccacbbccaaaaabbabbabbbaabcbaccababcbccaaaabbcacabaababaaabcbcabccccaaccbcacbaccabbbbcabcbcaaabbbccbccccabbcbccabbccbbacbbbccaaccaaabbcccbabccaaabaacbbcccbcbcaabaacabcaaccabcccabcbccccbcacccccccbbbaaabbacccbbbccabaababbaacbbbababcccabbcccabaccbcbcbabcbbbabacbbbabcbaaacccbbcbcacbccbcbabbcbcacaabbcbbbccacaccacbcbaccbbabccbabacabacccbbaabcbacaaabccbabbabcabbcaaacaabaccabbbbcbbcbabbcaabaccabccbabaccccbaacbcabbbbaaaaacbaaccaaaccccbbcbcabbcaabcacccaacacbbcacbbaccbbcbbaabcabbaabcbcbbcacbaacacacabaaaabbcbbbbacbbcaabcbbbccababbababccccaaccaabacbbccacbcabbaabacabcaccbbaaabababbabbbbbcaaacbbbcaaacccbccaccaabccaccccbbaabbbacaacacbccacaacabbbbabacbcabaaaaabbbcabaaaabaaaaaaccbbcabaccbccacaacccbaccabbbabccacababacaccbbcccacacccccabbaabacbcbcaabaacccabcbaccbbcccaaccaaacbcbcccacacababbacbbbababbbbacabaaaccacccbacacabcbacaacbbabbcababcccababcaccaacbbcaccabbaacbcbcbbbcabcccbcabcccbccccbabaacbcbcabbbabaabaccaacccbbbcbcacccbacaacaccabcaaaaabbabcacbcbcabaccbaacaacaabbcabcbabccbacacaabccbacbbbaaccbcbbabaccccaabbccccbaacabbcabaaaabcbbcbcaacccccbaabccbabacaabbbbbbcccbbcabbbcccbbaccaacabcbbcbccabbbcacaacaacccbbcaaaacbbbbbabcbababbcccccaabcaccaaacbcabbaccbccabbbaccaacbacabacbcacaabbbacabbacacbcbbbbcbcaacbababccbbbabacbabcaccbbcabbbacaccbbabccabacccacbbcbbcbbbcccbcbcbaabbbbacbbaabcabbbbababacaacabacababaaaccaabababacbacbcaacacbbbbabcccabccaccacabbacacccaabcabcbaaccacacaaccbbbaacbbbbbbccbababbabccabbbabbcbababccbbbbbccbcaaccbbbcbbccbccbbaaabacbaccbaabaabcbcbbcbbabcbcbbbabcccccaacccbbaacabbaacbabbcacabcacacacacbababcbabcbaaaaccacaacacaccbbacbbacccbacbaabbcaabbbccabbabcbcbccaacabbaacaccbccacbaacaccaccabcbbcabcbbabccbcaacbbabaaa","baacbccaccbbcaaacaacbacbababcccbcacccaacbcccbabc","abbccabbbaaacbacaabbbbabaaabcb","acbcbcbcabbbabacaccbcbaabcababaacbcaaaabaccaccbbcccaaaccbbaacabaabaabbaaaacbcbabaababaabccbacbbacbccbbbabbbacacbbbbbabcabcacabcbbbcaabcaccbacaaccbabbaaccccabbbcbbbccbccaacbbaacaaabcabaabaccabbcbbacacccabbcacbbccabcabcabbbacbaaaaaccabbbabcccccaacaabbccbccbbbcaabcbcbacacbacaabbbcaacacbcbcbabcacacbcbccabbaaccabcbcbcbaacbbbcbbbccaabcabcaabacbbbcaaabbbbcabcbaccbcabcbabcabccbcbbaacbabcccaaaabbacbaababbbccbcbbbcbbbbaabccbabcabbcbabbababcacaacbcaacccbbacbaaaabcbab","accbacbcabbcacaccabaaacacaaaaaabbbbbbbcabacaaabbcbbbbbabacacbbbbabacbabcbbbaacabccccacbcbabbbbbaccbcbbccbcaaccaabbabbaabccabbcbbaabaabcbbbcaaaccbbaacbccabcaaaabbaaabacaacaabbaaaacccacbccbbaacbaabbacabcbababbbbbaaacaacaacaacbabcbbcaacabbccbabbccbaabbbccbcccabbcaacbcbccbcaabaabacaccccccccbccbcbcbbaccabcccbbbbaabcabccaacbbaccccaccacbcacaaccaabbaaccaaaaacccccccabcbcbbabcccbaaacbbcaccbbccacacabaaacaaabacbccaacbcbabbbcbcccbbbabaaacbabcabbcbaaabcbbbbcacacbaabaacaaacacabacabcbcababaabccaacaababacaabcaccbabaabbbcbbaabacbcbbbbbccccacaaabcaabbabacacabbbbbababbcaaacbccbbbbccbacbcccaccbcaabbbabacabbbbabcbcbacacbcabccbaabbaccacbbcaaacabbcaccbccbabacacccacabccacaacabbacbccabccabbbccaccccacbbbacbaabbbbcbcbaaabaabaccaccaccabaaacaacacccabaacccababacabacbbacbbabbaaccbbcabcbbbaaaccaabaaacbbccaccaaaabcaaccaacaabccababbccbcaccaccbcabcccaccbcbbabcccababaacaabcccbbcabbacbabaccbcbaabbccaaabbaccbcacbabbbbcbabcacbcaaacbaccbcbbabbcaccbbaacbabaababbabccacacacbaaaabaccbbccaabbcbabacacbabbccbcbcbacbbaabbcbaaabbbcbcbabccabaababbbabcabbcbbabacbbabcaaabacbaabbcabbbaababaabbccababbbccccbbbcaacbabcbabbbbcacbbbbcbbcbacbbcacbbbbcbabbbbbaaccbacabaaacccacaacbaabbcbacabbbccabcaabaabcaabccccabbcbabccabbbbccabbcbcabbcbcbaacaacaccccccbbccbbccacacbcbaacbaabaababcaaaacbccccaccacbbabcbccacccbcccababaccbbcccacbbabbccbacaabbbacabacccaababcbbcbcbcaccbcbcacaababbaacaaabaaabbbcaaaccacbccccacaabacabaabccccbcccabbcaaaacabcacbbbabbaababbccbbcbacbbbbaaaccababaaacaccaccabaacbacabcbcccbcaabaabbacbcacccbcbccaccbccbcbcaccccbbbccaccbbacbbcbacbcbabaacababccbbcacbcaabbbccb","bbacaccaabbcccabbcbcbcaacbbcbbbcbacabcabcbbabcccbaccaababccccabccacabbbacaabacacaababcccaabcaaaabcbabbccaababcabcccaacaabaabcbbacacbcabaacbcbbaacacbabbbbbcbaccccbabccbbbcabbcccaabababcbbbaabcccbaacabcbbcaabbbcbbbaacccabaaccbbbbacccbbcbaaccaaacccabbacccabcbcaabbcaacbacccabbabcbbcbbacaccabcbbabbbccacaaabcccbaabccaabacaaaacbbcbcbaacbacbbabcabaaaaacacbaacaabccbcbbabbcccbcccbabbcbbabbcacbcabbbcaaccacbaccbabbbabcbabacbbbcbbababcacaacacccbbbabcacaabbabbbaacacbabaabbabcbbccbbccacbabbcaacabbbcabccbcaaccaabababccbaacbcacacaaabccbcbbaabccabacaacbcaacbacabbbbbcababcbcabbcbcacbbbaaabaaabbbaacbcaabbbbbcacababacccbbaabccacbccbcaabbaacbcbaabcaabbabacbcccaaccbbbaabbacbaccaccacacabbbbacaaccabcccacbacabbcaabbbabbbcbabcabaccacbaaacacacabbabcbccaccacacccbbaacaacbcaabccaacbaccbcbbbababcacacabaabacababcabcaabbbbbaaacaccbacbacbacccaccabcbaccabccbccaaccbbbaaaaccbaacccbbcaccbcababacaabacbabcaabcbcbababbabbcbacbcabacbaaccbabcacbbbbcaaccaaccbaccccaaccabccaabacbbcbccbcaacabcbbbbcabaacacaacaacbaaccbbbbbbabaaabacbccbcccbbbcacbaccbabbbacbcbccaabbcabaaacccacbccacbbbaaaabbacaabcacaacbbaabcabaccbbaacabcbbcacabbcccbbbccaaaaccbbbaabbacaaabbbccbbaaaaacacccccaaaaacacbcaabaabaccbacacbaabbbacbcbc","cccbbb","abcabcbcaabcabbcbccbcbaabaccbcaaacccbababbaaaccccaababaacabbcaaabacacabbbbcabacbccccbccabccabccacacccabcbabcacabbccbcbcaaaacbcbcbccacacabacbcbaca","acccbbc"]
b = ["...","cbaacbbcccaacacbbbbbcbbcccb","bcbaccaccaaaabcbbbacaacaabaaaccacabaaacbcababbacaccaccbcabbcacababbbaabcbacbaacbabbaabbacacbcbabbbcccabccaccbaaccbacbacbcccccbbbcababbccccacccccbccbcbbbcbaaabcaccbacaabbaacacbaabbccaccbbbbbacbcabacacabcccaacbbbabcabbabcaccaabacabaccbcbbcabc","cccabbabcabccbbbcaccabaaaaccbbaaccaabbcccababaaacaccbabbbccabbcacababcacabccbccbbbbccabbaaabcbbcacaccaccababbaabacaaaaabbacbacaccbbabcccabacccabbabccabacacaaccabcbaacaaaccbccaccacbacbbbccbcccbbbababccccbccbcaacaabcacbbacabaabbabbbaccabbaabcaabbbcbcaccbaaaaaacbcababbcbaaaabcacccbbbaacabcabacabbbaabaacaccbccaaaabccbbaabccbbbababbaacbcaccbabbcacccbbacaccabbbbbbccccca","babbccabcaabcaacbcaabbbbbababbbcabbcabacbbbbabaaaaabcacacbabcacccaacccbcaacaaabcbcbbcaccbbbaacbccbbacabacabbaaacbbabbaacbbbccbacabbbaabacccbcbcacacabca","aaaaacaabcbabcababaacaccbcabbaabcccccaacbcaacbbabacacccabbabaabbcaaccacabcacacbbaacbaabcbbbcbaacabaacbaccccbbcacbcacbcbcccbcacababbabcabcbaaabcbcbbcbacbcabaaabbccccabaccaacccabcaabaccaccababcbaaabcbaabbabcbcaabacccbabcbaacbbcbbaaccabcbbacacbccbacaaaaaacacbabbccaccccabacaabaaccababbbaaaaabcacbabbaacbcacbccbacbcbcaccbaaaccacaaacbaccaaacccccabcbcabcaacbbbcabcbabcbbbcaccccbbcbbccabacbaacccbacccccacaabaccabcc","ccaabbcbbccbcacabaaacbccaaabcaabbcbababbbabaaabbaacbbabacbaaacacaabbbbabcacccbaabaaccbaababbbbaabbcbabaccaacbacabaabaaabcacaacbcaaaabcbcaacbabcacbbcaccaaaaacbcaabcababaabbcabcbbbbabacacbbbaccabcbbcabaababcbacabbbcacaaaaccbcccaccbbacaaccccaccbbbacacbaaacaccbaccbaaabbcacacaaacbabacaabcbbcab","aabacacbcbaaacbaabaaaacacbbaacbbcaacacbcababccacaacaaaacbabbacbacbbbccacbacbaaabacacaccbaaccaaacaabbacabababacbcbacbacbbabcaabcabccbabcabaaaacccccbacbaacaaacbccaaaaabaacaaccccccbabcacbbabaccacbcbaababcaabcacaabcbcbbaabbabcaabcbabbbcbbabaaccaccaabcbaaaaacccaabcabcbbabacabababccbaacabcaabaacbacaacaabbbaacbabbbbcacccbabaabccbbaacaabcacbacbcccaacabbcababccaaaabcbcbbabbbccbbaccbaabbcaaacacbbabcccabcbccacaababccbccbacabcccabcaccabbcbabacaccaacbcabcbbabaaacbcaaccaabaaacbbcabaccbcbcaccbbacccccabaaccbbbbabcbcccbbbbcabbbbcbbbacacacbbaabababbbbbbcabaabcaacbbbbcbabacbaccbccccbccbabccbccbacbabaaaacaabccabaaaacbacccbbbccacaacbcbcbacbcbcbcbcbcbcacaabcbabcaaaaabbabbcaccabbbcacabababbcacaacbbbbbbbacaabacabcbacbccbcaacaccccbbaaccbbccbcccaabcccacacbbbaabbcccbbbcabbcabcbacbaccccaaaabcaababaaaabababaccaccbccccacababcccabaaabcaaacaabccccacabcaacabccbbcccbaacbacccbbbbcbcaccaaccbbcaaababacaccccabcccaabaacbbcaabcbcaaccabacaccabcacabbabaaccaabcaaaaccacacbacacabaabcacccaababcaaaaacccbaccbbacaabaccaccababcacbbaabcbbaacacbaaacabaccccabbbbabcaaaccbbbaacbabbcaacccbcbcabbcabacaabbbaaaaacccaccacbccaacbaccccbccacbcaccaabbcabaacbccbbacbaabbbabbcaaabcaccbbbccaccbcbcaaabcaaabababbaaabbbaccbbcbaccaccccbbabaacbccbcacacccbbabccbbbbbbaccccbbaacacaacbabbbcacbaaccaacbccaccccbbcacbbbcabbbbbabbabcccccacaacbbaacaccbccbcacabacaababbbccabbbaccaccaaaacbacaaaaabbaabbbbabcaacbabbcbcccaabbbbbbcbaacbaabcacbaaabbabacccccbcbcaacccacabbbaaaaaccaacbccbbacbaacabcbaabbabbccaaaccabcabcacbccacbacabcbccbccbbabccaaaaaacbbbaabccccccabbbcbcaabbbcbbbbbbacabcccababcccaacaabbbabcabccbaaaccbccabccacccabbabccbacabbabbacaaaaabaccaaccaccaaccbcbcaabaaaaabbbbbbccaacbabcccccccababbcabbccbaaabcccccacbacacaaacbccacbaccaccbbabcbaacabbbcbcaaccacabcacbacaabbbcccbcbaccaaaaaacacababacbbbcacabaacbaacabbacaabaacababbacbcababccacaababacaccbaccaccbabcacbcaabccbbcababcbabccaacbabbabcbacabbbbaaaacbbbccba","abbaaccaabcacacbacbbccbbcabcbcacbacbabbbcabcccabaabaabcbbabcaababcccacaaabacaabaabccaaabcbabcbcacccbaacbcacbaaaaabbccacabbbbabcbabcbbcbabbabcbaccbcccabcccbbbbabcbbcacbbaabcbbacbcacabbaacaacaacaccaacabbbbaabbcacaaabaacbcccaccbcccbcaaaacababcacababcabbccaacccbcbbcbabbbccbaaccaabcbcaccbccccabcbbabcaaaaabcbcbaaaaccbcbabbbbbacbaabcacacaabaccccaaacbcababababcccbaacbbaaabccabaaaabbbbbbaabaccbaacbcaaabacaabbcacbcabccccbacabbabbabcbbbbacbcaabcccaaaabcccbbcbbabacccccbacacaaaacbbbaabbcbcbcccaaaccbaabccbccbcbbccabacbbacabbaccbabababcaababcaaaacbccaaabbbcbccbabcabbbbbabaababbcaacbbbabbbacaaaabcbababcbacabccacaaabbbacccbabaacaacaccccaabcbcabbacbccccabbabbabbacbccaacccbaaaaaaccacaacaacacacbbccbababcbccacccbbccccbbcccbbcabacabbacaababababaabaabcabaaacbacaaabaabbabcacbcaaaccbbcacaaacabccacbcaccbbcbabccbaacbaabbbbcbcbbbbacbaaaacbaccaaaaccaaacacbccaaaacbbaccccaccbbacaacacbcbcbcccbaabbaabbaabaacaacaabcccaacbbaaccccaabbacbbcacbccbabcbcacbaabbcccabbabbcabcbccbacacabbbcbaaccbacacbcccbccbcbccabaaaaabcccaaababbcbcbaacbaccaaaccacbccbccacbacabacccbbbbaabbccacbcaaacbbccaabcaaacbccbccacbccbcbaccabaaaccbcaaccbabcbbccbaacaaaacbabaaccbaaabbbcbbbcbaabaabbaaaacabcaabaccbaaabbaaabbbbccabbcacbcacbbbacbcacbccbbbacabacccccaccaacccaabccbcaccabaacacacbaabbacbaabbbabbaabbacccbbbbaacbbacabbacaacccaaaacccbbabbbabaccbabccbaabcccacbbcabcaccaaccacacccccacbbaacbcbacbaccababcaacbabcbcaaababccabaaabbccbcbbbaccbbbbccccbccbbccbcccaccccbccccbbccaabacaacbbccccaaaabbccaacbccaababcccaaaaaaabcbccaacccbcacbccaccaabbcabaacabbabaaabbacbcbbbcabcbbacccbacaccccbccaccbcababbccccbabccbbcaaacaccabacbcbaabbaccbcabbcbbbcaaacbbabbbbccbbcbbbbcbabaacacbcabacbbbaaaaaaaabccaabbcaccbabcbcacbacbcaacacccbcaacbacbaacabbcaabcabaabccaacababbaaaccaacabccabacaacbcababacccaacacabcaccbaabbaaacbaccaabcbbbbbbbcbabacccacaabcaccbacccabacbbacaaabcbbbbbacccabacaabbcbbbcacbabbaaacacabacaabcbbcccacccabcaccacacacbaabbabcbbabbb","bcaabbbabaacbcabccbabaccbabccbaacccbabbcbbaccabccaccbbaacbbbbbabcbaccabacabcbbababbbcbcccaabccaabbccacaabcbcccbcabcbcbcabacaacbbabbababcabbcccbcacbbcbbcbccbccbaaaabaccacabbbaabcacacacbacbc","cccbcccabbaacabacacbbbbcbaccccacaacabbabcccbabcabcbbaacbabbcbbbaabbcbcbccbacbaaabbcbaacbbbababcbaabccacccccbcabccbbbbbababbabcbbbcbaabbbccbacabaccbcbacbaaabccbcaccccbcbbccbcacbcbaacacabacababcaabbacaaccaacabaababbbcacbaacaccbccbabbbcacacaccabcccbacbbbbcabcbcbaccbbacbbacabccacbcbaaacbabccccbbcaaacabbcacacbcaccaaaabbcabcbccacccbacbabbcbcccaacbbabcbcccbaabaccbbcbabcbccaababcbabbcabaccaaabaacabbacacaacacbaacaaabbabcccaccbabbbabcaccabaccbaaabaabaaccbbccbaaabcbbcbabbbbcaacabbbaabacaaccaacbcaaccbbbbaababcccaacbaacacacbcaabbccbabcccaaabbacccccccaaabcbacabcacbabcccbacbbcbcccaaaaaacccbbaacacabcaacababcabcaaacaacaaccccacacccbcbbccbcaabcababbbbccabacbcbcacbacaaccbabcbabcbbaccabaac","bbabbbbccbbcbaabbbbbacacbbacbaabbcccacaacbacbcbbbcbaabacbbabbababaabaacbcbcaacbbacaccaabaaabcaacbaacbacaccccaacccacbbbccacaabcbaaabacabcbaabbbaaaacbabccaaabaaaaacccabaacbcacaaabacacccccbbccccabccaacaacccbacacabcbccbcccacabacbcccbbaccbcabbbabccacbbbbbcaccbbcabcbbcacacaaababacacabcbaaaacaacbbacbbbabbbababaccbaacbcabcbcccabcbcbcacbbccabaabacabcaccaabaabcacbcccacbccacbabbbabaccaababbacaabbaacabccbcbcbaabcbacbabbbbccccbbaaccbcbabbacbcab","bbbabcccaaaabbbccabaabbbbacbbcaaaccaababcacbbaccbaaaccacaacbbcabbbcccaababbbccaabaacbcabcbbbaaabaabbbaabaaccccabacbbabbaaaabaacbbbbaaaccacacaaabbaaabccabbacbcbcccbccabccaacccccbabbcaaccaccbacbbaabbbaacabbcbabcabbbcbacbaabaaaabcbabccbbcbaabacabbbbabaacacbbbaccbbbcbccacacbaabbbbcbccbabcaccccccaacabcbcaabacacaccbcbbaccabbcccbbbcbbcabbbabccacbbbaacabbbaaacbbcccbbaccaccabbccabaabacabbaacccaccbbabaaaaccbbacacbbabcbcccaababcbabbccabbabacbcaccacabacccabbcbcbcbcaaacabaaaabccaabcbbaabbbcccbacccabbbaccccbbcacbbbbccbbccbacaabaacbcacbccacaaacbaaaabbcccaabbcabccbaacbacacaacbabbbbcbbbccbabbbaacabacbaabacaccbbccbbacccababccbaabaacaababcbbbcccbbbbacbcbcacaaacbcbacaacccbcaabbccacbbbbbbaaacbacbaaacacaaabcbcccccbabaabacccccbcbbaacaabcaacbaabacacacacbaaaabcbbaabaccbababaaacbaaacabbbcbaacabcbcabcabbbbaaaccacbacccabcabbabbaacbcbbbaaabacbcbabacbacaabbacaababbbcbaacabbcaacaccacbacbcccbbccbbccccbbaabbcaccaaabcaaabaacaaaccbccbcbcbcacabacaabbbacbabaccbcaccacabbcccaabcbaccacccccbabcaaccaababccccaaaabcabcabcccccaacaaacacabaabcaabbcacababacaaabacbcbcbbaccaabaaaaaccbbcabccabbcbbcaccbcbcbbbcbbaaacccbcbbcaabbacbcccabbbcaabbabcacaaacabcbacaababcabbbccccbaabacaccbbcabcbbcacaaaaabbaaacacacbcccaccaaacbbacbacabaaaaaaacaaaabbabcbbcbacbabbcabaababbbcaaaaaacbabbbbaacacbababcbbcabbaaacabcbcbcabaccbbcbccbcbbabbaacabcaaabbacacaacabbbaabcacbbaacaaccbabaaaaacacabccbaacbccaabcbccbbaacbbbcaacaaccbbbacaccbaccbaaabbabbbaccaccbcacbbbcbabaccabbcbcbacbacabbcaaacbbaaabbaccaabcbabcbbcccacacbcbbbbbccbbbabbabcccabccaabcbcccccabcccbbaaabcbcabcbbbacabaccbcaaaaaaaacabaabcbabacbbcaabacacaaaabbbaaccbcabcbbababbbbbaaaaabcacbccaaacacabacbbaccaabaaacbbcaaccabbaccbcacbcbbbcacbbcaabccbcacbbbcbbccbaaaaaccabbcacccaacacacaacacbbcacaacbbbccbababacbcabcbbbacbabbcababcbccccbbbbcbacbabbcbcaaccbccbcbaabbacabaabccaccaababcacaaabbcccaaaabbcbbbcbbabaccbccaabbbcbabbacbccccbbaaacacaabbaccccbccbacbbabcaaaaaabbaaacaacbaccbcacbbcbcccacbcbcbcccabbbccccaaababbaabaaaabccbbaaaabccabaabccccacacccabbbabbbcbacbbaacbaaccbabcbcacaaabbbcaabacccaacbbbbb","babcccbbacbcacbacbaaacbbbbaccbacbaaccababcbbacac","cccbababccaabcccccaaabbbcbccbc","cbbccacacbcabccabccbcacacaaacaabcacababccacaccaaccabcabacbbabcaaacbcccacbcbcabbcacabbccccccbbabacacaaacccaccbbbacbabccbbbaccccbcbccbabababbccbbbbaccacccbabbcbabccaaccbcccbcaccbbbbcabcacaacaccbbbbabcccccabcaaccbacabcaacbaacbcccbcacccbbacbbababbacacbbcacabcbbabaccbacaccbaabbaababbbbabacaacabcaacacaccacccbcaacacbababaabbbcacaaaccaabacabaaaccbbabacaabbaccacabbacabbbcbcbabbcaaccccaacccbbcacccbbbaccacaccabccbbbaaaabbaaacbbabacbbbabbbaabacccccbcbababbccccaaacccbb","accbbbcaabacbcabaaabcbbacbcabbabbbbccbacaccbcbbbccbcacbcbbbacaaaacabbabcabaaccabcabaaacaacaaccbacbcabbbbabababbccacbcbccccaacaabacabcacbacaaabbbcccbbacacbaabaaababacabcbbbaccbbcacbabaabbbabaacaaccabccabaabcbbbbbbccbbbbbabacaacccaaabbcaccaabcccbacabbabcbbabaabbccccabaccbcacbcbcbacababbbbbcaaabaabbbbabcbccbbbcacbabacbccacacbaccacccbbaccbabbbccbccbbccbbbcbbaaaccaabacbcbccaabcaabbabcbcbcabaccbbbaacaacbabcaacaabccaabbaabaccccbcccccccacaaabaabccacccccbacaccbbaaaaabbcccaaacccbcbbbbaacaccacaabbcbccacbcaacaaaabacbbacbabacabccaccabbaaaabbacaaabcbacaabccacbbacaabacbbbacacacbcbaacbcbacacbbbabacabbccaaacccbbcbbacbbccccaaaaabcbaabaccccbbcbcbbbccbaaaacaacababcabaacaaccbcbacaccabaabbbccaabcabcabacaacccbacbbbbbbccaaccbcaaccaacbcaccbaaabcbaaccccbbcaccaccacacccbcbcbacacabcbaababccbbbccbbacbbbccaaccbbabaaabccccacabbbbcaacbaabcabbcacacacabacbaccacababacaaccabcbbaccbcacccbbbcabacbcbccaaacbbcacaabcacccabccaccbaccccbababaccabcabcabacacaaacccabbaaabcabcbabbaccbbabbcabcbcaacccccaacabbbacbccccbccbcccabccaacaccbaaaaaccbbaabbababcaacbaabccbcbcbabaccbaacbaabbbbbcbbcbcccbcaabcbcbcacabaabccabacbabacacbacaaababaacbcbbbacbaaacbacaaaaaababaabcaaaaabcaaccbaabcacaaaaccbabcabccbbccbabaaabbbbbccbbbcabccbbbaacaccabccbcbccbabaaabaccbcbababaaabcccaccacccaabacbbbcaaacbcaccccabcccccccccabbcbbbaacbbacaaacbcacbcccbabcbccabacbbccccaccacacacbccacacbcbbbccccbccabbaaababaccbabbbabbcaacaccccbaaacabccababbababbcccabccbccbbacbcabcabbbcccbacbcaaaabbbccbabaccccaaaacbabbcaccbbacbbcababcbbbacacaabbcbbbbbcabcbacbacbbacacbcabcabcbbbbabccbbabcbbabbcacbacaabaaabacbbbabbb","aacbabbcccabacbbbaacabacbcaaabacbcbbcbbcabcacbbcacaacabbccbbacabbbaccbbcbbaccacaccccccbcbaabcbbbabaacbabcbabcacbbbcaaaabbacbcbacabccbbabcbbccacacbbbcbcccacacccaaccbbabbcaacabcacbacbccaccbbccaacabaaccaccabaccaabababacaaacccccaaabbaaccbbcbbbacbbabacbbccaacabaaabbcbcaaacbbbcbcabcccabcabccacacabbaccccbccbbcaaababacccbbbaabbababccaaaaabacbcccbaaabbaaccacacacbbcacbabcacbccabbcaacaaabccaacbcbaccbaacacccccabbbcabbbabbbccaccabccabaccbcbcbcabbaababbbccaccbbaccabbcaccaabcbbcabbcaacbaaaaaccacbcbccbcacabcacabbacabcabacacabbacacbaaacaaabcbacacacacaabacbbbbcbcababaabcabbbbcacbcaabcbaacacccbacabccbabbaaabbccbaccaaaaaabccacaabbcacabacaaabcbababbcaabacbcacbcababcaccbbbaaaccaccbabccbacccacacaaaabbaaaacbaabacbccccaccaaabbbcbbaaacbcbbcbccccccacbcccbcabaaccccccbcbabcacbbbcaaacccacbaabccbcaacaacbaccacbaacabbabbbcaacacabccbaabacbaaabccacaaacacbbaacaabaabcabcbabcacbcacbababaacbcbaccaacbaacacabaccbacabcaabccbcaacabccbabbccacbacaabccabbcbcacbcbaccaccbbacbabccbaccaccacababcbbcbcaabcbcbccabcbbbaacacabaacaabacacabaaaabcbbccbcccaccbbbaaabcacbccaccaccbccbababacbbabbbacabccbaabcababcaccbaabaabacbccaaacbabacbcaccbabaaccacbcabcbbcbacbbacbcbabacacccacabaabbabaccaaabaabbbcbbacbbaccbbccaccbccb","caaacc","babaaabbbbbacbbbccbcbabcaaabaaccccaabbcabcabbcccccaccacbbccaccccacbcbccabbcbaabbbbcccababcbcccbaacbbbbbacbaccacbbabcaaacbbccbaacacccbabbacaabcbba","bcbaabc"]
s = "aababaabaabcbbbcaabbaaccabaaabcccaaacbcacababcbbbcccabacbacaaccacbccccbbbcbacbbbcccaabbaaccacbaabaabbbbcaabcbabcababbaacbbbcacaaccaccbcabbababbaaccaacbaaaaabbabbaacaaccbbcbbbabbbabcbcbaabbabcbacccabbbcaaaacaccbbcbcbcaaaccacabacabcacaccbaccccccaaccbaccbcccabbcbbbaacbccbbcbbaaacbcacccbbcbbbbbccbaaababcabababbbbbacbbaabcabaaccbcababbcacacaaabbabcbaacaaabaabbaccbaacccbcbbabbcacaababcbcaaaccccaabbccacbabbbacbbcbccbaacaabcbacbacbcaccbbccbbaabcabcbbcbbcbacbacccbcccbabcbbcacccabcbcbcababbaaaabbababbababaaabbbccccbbbcaacaccbabaaabbbacccbbaabccbbcabacabbbbabccccaaaccccbccccbaaccbccbbcabbbcabaaaacbabaaabacbaabbbbcbaacabaccaabcaacbacccbabaccbbbbaabacbbabacacbaacabbbaacacccaaabccbcacccccaaabaccacbcbcbcabcababacbcccbcabbcaabcaabcaccacacccabcaabcbbcaccacbcacbaccbaabccababacaabaacbcacccacbcacabbbacaabababcbbbccabacabcabccbacaaaccaabbccabbaabcbaccacbcbaaacabcbacccabccbbbccabcaaccccbabccbcabaaabcaacabccaacacbcabbbbbbabccbbcccabbbcabbcabbacbbabbcbaabbbcabbabcaaabbaabbabaccacccbbcabbbbcaabaccabaccccaaabbccbabbbacaacabcccbaaacbabaccbabbccccbbbbaabbabcccbcacbbcccabaacacccabbcbbabcbcacaabcaabbcbcccaccababacabcacbbccccccbacccaaaaaacbbcabccbbbbbaaacabaaaacaaaacbaacbbabbbacbbcbcbccbabcbcbacccbcccabacbabbbbabcbbbcbcbaccbbbcabacaabaccbaacccacbabccbbbbcbaacccabcaaacaaccacaaccbccbabaacaaaacaaaaccaacacccbcbbcabcbcacaccacabcaaaccaacbaaaacbcabbbcaccbcacbbcbacccaaaaccababbaacccaccaacccbacbabacbcbcaccccaccbcbccbbbabbccbaaccacacbaaabbcbcbccbbcbabaaaccabcccccaabbaabbbcbcbbabcaccccbcaccacbcbabaccaacacacabcbbbaabbaacbcaaccbcaacbccbaaacccbbbabbaaccacaacabbabbcbacabcaacbcbaccabcbaabbcbcbbbcababbaccbcbccbccabcbabaaccbbabbbccabaaabccbabcccabaabccccacbbacaacbaaabbbaaaaccbcaaabbbcacacbccccbbbabbcbabcbcacabaabcaabbbbcbaabbcabbaaacbbbcacbbabcbcbbcbcccabacbbabbbbabcaaabaccbcbbaacaabacbbcbabbbccacbaabaabbbbaccbccbcacbacbaabcbcccbbcbbabcbaacbacccbaaabbbaaacaaccacbbcbcbbccaaabaacacbcacbbaaabaaacacccccccbccbacbcaabaababaabcaacbcbbaaabcbbbabcaccaaccbbacccbbcbcbcaabbbbbacbcabbbbaabcbcaacabacacbaccaacbbbbabaacbaccacabacbababccbcabaaaabccccaabcacccaabcccacbcccaaaaccbcbacabcbacaacbaccbbbcbbbabaaabcccbbbaaacabcbaacccbbbbababcbcbccaacbbbbacbcccbccbccabacbabccaaaaccbacbbcabcaccaccbcbcbbaccbacccaaaaccbbcbccbbaabbcbbacaaaaacbacbbaabbcbaaacbcacabbcbccabacbbaaacbbbabbbbccacacbacaccabcabcacccbccabbcaccabcbccabbccaaccaaabbbbaccbabccacacaacccaccbbcabbababcabaaccacbccacacbbcaccbacabcbcbbaaaacbcbabaccaababcbcacaabbcbcabacabbbbabaaacacbbaacaccaacbbbaaabccbbacaaccacbbcaabbcbaacbaabbaaaabaacaacaacabacbccaccccccccbacccbbbacacbcbaccaaccbaaaaaaaaabccbaacabcaacbcbaaababccbbaabcbaacbaccacccabcccaccbbaacabbcbbbbcbabaccbcabbabaccabaabcbbccccbabacbbcaaaacacbabbcaacbcaccabcaccccccbccbbacacbabacbbbcaababacababacbbbacaccbaabcabcacccabcaaaabaaaacbbabbcbabcaaaacabcabbbbcbbbbcaacbcbabaacccbbbcbbabacaaababbaccbbbccacccbbccbcaaaccabbcacccaaabacccccaaaacbbbabbacabbbbbcbaacbbabcacabbaabcbaaabbbaacacbaaaacccbcabcababbbaaacaaccbcbbbbbbababbbccbaccbbaaacccccabcbbcbabcbbaabcabacbaaaaacbcbcbcccabbaaaaaabaccaaacbaaabbbabaacacbcbbcbcbacabacaabbbccbccaaabcbbcabaaacbbabbabcbbbbbaaccbcaaabbabaaaacaaaaaccaabbaacbacbaacbcbaccacabcaaabacabaabbaabacccccbabababbcbcbaacbbaccacbcbcbbabcabcbcccbbabcbbbccabbabbbbcacbcaaabccccabaacbababcabccabcaabacbaccaacccbacccacbcbbcaabaabcbbbacccbbbaabaccbbaccbcacabaacaabbabcbbbaacbcacbcbccbccbcccbbbbaabcbbcbbcbbacaccbbbaabbbcaabcabbbbabbaaccbbcccacccbcbcaacabacaccaaacbaacabbcccaabacacbaaacbabccccacaabcacbcaaaabcacaaabaacaaabbbcaacaccbbacccbbabbbbbcbcbccaccccacbbccabbabaaaaaccccbbbbbcbacbcccabccbacbaabaaccbacbacaaccbcaabbbbcccbcbabcbcbcbabbaabcbbabccabbcabcacbcaaababacbcbccaacaaaaaccbabcaccbaaacccccacaacaacaaabbcbacaabbabaaccabbcbcbccabccbabbacbbcbbccbcacbaaacbaabababbbccbcbcbccabcbcbabbbbaaacbbcaaccaaaccaabccaacbbbbacabbbcaccaaaaaababcbccacbbcbcbcabacbbccabbacccbcabaabbbbbbcabbaccabaabcabcbbaaaaaabcaaccabaababacaaaccaaccacacbcacacbcaabaaccbccbcababbbcaacbcbbaccbccccbcbaccbcaacacacaababacacbbabcabcbccaccaccaabaccaaabcbacbcbcabcccacacbabababccabcccacccbbbbccbcbbcbaabaacbbcabacbbabcaccacaabbacbababbbcbcccccaaacbabaaabcaccacbaabcacabbaaabbacccacaccbacccaababaacbabacacbaabcacacacbcabcabababcacbcbabcbbbccbcbbbccaacaacbbabcacaacccacbcccbabcaabbbcabbabccbacabbccaccccabbcacbabbbbaaccaabbcbcbccbaaaccacbbaacbbbbcbbcabcabcbbabccbaabccbaaaacccacabcccccbacacacababbabbcccbaaabacbaaacacacbbaabaaababccabbbacccbcaaaaacccbbcabcbabbcbccacacbacbcabbbabcabbabcccababbcacaaabbaaacacbbaccbcababaabbbcaaabcaacccbccabababaacbbacbacbbcbcccbbcbbbccababbabcabbabaacbababaaacbbaacbcccbbcbbacabbacaaaacbaabbabaaccbbcbaccacbcbcbccabcacabbbabbaabcabaacbbaacaabacababbbbcbbaacaabcacbcbccabcacabaacbbacbcacacbacbacacabbaccbacaccabbcbbaababcaaabaccbacbbcbcbabcbcccbccabbacabcbccacbbabbaaacaccabcacbbaababcbabcbcbbabbacbbcbbacbcbabacabbacaccccacbaaaaccbcbacbabbbacbccccacbaabcaacbccabbcccbacacccabccbbbbbbaccbaacbbcbacccaaabcabcbabccbbaabccbbbacccccababbabcbbcabcccabacbaaaccabcbbbbabcccccaaccbcaaccbaaabbcabbcacacbcbbaacabcaacbaacbacacccbccabcaaaaaaacbaabbbbcaccbbcabbabbcbbaccbabcbbabacababcaaabbaabacbacabccbcbcbbbccaccabbbabbaaabcaaabcaaccccbcbbccccabcbabccbccbabcbababbbccbcbcaaccacabbabcaabcbbccbacbacacccbbcabbabacbcababbacababababaabcacbacacacccbccbbabacabaccacbababbbbbaaccacbaccbbbccbbbbabbababbcbcaccaacaaccbcaaaacacbcccccaacabcabcbbbbacbbbabcbabbccbccbcbccbcaccccaacbbabaaacbbbcbbbaccacbcacacaabacbcbccaaccbbbcccaaacbacaacccacccabbabbcbcaccaaabaacaaccbbbcccacbbacccabbbbaabbbcbbbbbcbcbabcbaabbbbbacbbacccbabaabbbbaccbbbbccaccaacbbbcbabacbbbbcbacabbbaccacabaabccababaccbbbaabcbabbccaabaaccbacaabcabcbbcbcccaaacaabcbabbbababcccaaaaaacccaabcbbcaacbabcabaabcccacbbcacbcaabababcacccaabbccaabbacbababcbabbcccaacbcbcccccabacaabcabcacaacaacaccbbabbcaabbbcbabccabbaaabcbbcabaabaabaababcaacacbcbcccacaccabcbbaabbbacabcbaaacbbbcbababbccacbccbaaabcacabcbbcbccababacabcbabacbbbcccbcbaaccaccbbabacbbbacbbccabbcacacccbbcbabcbcbcbbbabcbcabcbbacbaaacaacbbbabbbbbbaabcbbabcbacbccbbbaccababccaacbccaaabbaccaccacbbbcaacbcabaccaccccabcbcaaaabbbccaccbbbbaaaccbbbbaabcbccbbacbbcaaccbcabbcaaabaccccacacbcaaabcacbabbccbccaabccbbcccacaaaccaccbabaaaacaabacacabbbbcbbabbccbbaacbbaaccaacbacbabacabcbbbacbcacaaccccbabccabcccacaabaabaaabacbbabbbcbaababcbcacbcbbbbcaaabccaaabaaaaaaccabbaacbcacbaccbaaabcbabbccbacbbccacbacbcccaaabbcaccabbacbccbaacbacaacbccbbcbcabbabababcbcbbcbaccbababcbaabcbabababbccbbccbcbcbccbcbacaccbacccbaaccccbbcacccbccccccabbccabaaaaacbbacbabbaaccacbacbbcbbbbcacabccabcaabbbacaacbabaaccbaaaacccacacbcaccbacccababcacabbabaababbcabbbabbaccbaaacccababbbabaababacccbababacbccaccbccbacacbbbbcbcaacbabcccbbcaabaaacbbbccccbcaaaccbcccaababbbcacabcbcabcaabbaacacbaaccabbbcbccababacbcbaabacacaacaacaccacbbccabbaabbccbccccbcbcbaccababbaabcbcababbcabcacabcccbaccacccbccccbccbcccbcbcaabbbcabcbacbaccacaabcccbacaccbbacccaaacccccccbcbaacbccacaaaabcaacbacbcbbbaccbbcbbbbbbbaaaabaabbaacbcaccbccccbbcccaaabacccacbaccaacacaacbccbbcacaaaccabbccccbbaccaccccaaabaaccbabcacbbbacbbbbccbcacaccabbcbbaccbabaacaaacccbcbbacbacbaaabaacaacacbcacccbabcbbbcaaaacbcaaaaacccbcababcacbacbcccaaaaaaacbcbcbbabbabcbacbcaaabbcabbbccbbbbccaaccbbbbbaaacbacccbcacbcaaaccccacaacaabcccbaababcaacccbcbacacbcaaacaaacbcbccbbabacacaacababcaacbcccacbcbbcaacacbacbbbababacbcacacaacacababababccabbbacaacbcabcaacbbcaabbccbcbaaccaaacaabccababaaacbcbaccaababacbabbcacbcccaacacabcbbcabbcaaccbbabccbcbacaacbbaabcaccbccabcbbaacbabbcaaccbcbcbbcbbbaacabaaabacabbacaaabaabbbcbbbcbabcababaaaaababbcbabbcccaabbabcacbbacaccccbcbbacbbcbabacabaccbababacbbbcccbabaccabbbcbaaabcabaacabcacacacaaacaabbabacbabcacaaacbaacbabccbabaccbcaaccaccbabaaacaccbcaabcbccbcccbacacbabbcacaacbbcbbcacbcabccbcaacbacbcbabbbaaabaccbcbaabaabbbbbccbbccccaaaaccbabbcbbbcbbabbaccacccacaacccbbccabaabbaabaabaabbcbcccabcacbccccbcabacababcaaacabbbaabacaacbabcaabaabbaaacacbcbabccacaabaaababccacbbbbbcaabcabbcaacabaccacbbaccbcbacbbaabaaaca"
print(solution.stringReplace(a, b, s))

hash code equal, but str not equal
hash code equal, but str not equal
hash code equal, but str not equal
hash code equal, but str not equal
hash code equal, but str not equal
hash code equal, but str not equal
aababaabaabcbbbcaabbaaccabaaabcccaaacbcacababcbbbcccabacbacaaccacbccaaacccbacbbbcccaabbaaccacbaabaabbbbcaabcbabcababbaacbbbcacaaccaccbcabbababbaaccaacbaaaaabbabbaacaaccbbcbbbabbbabcbcbaabbabcbacccabbbcaaaacaccbbcbcbcaaaccacabacabcacaccbaccccccaaccbaccbcccabbcbbbaacbccbbcbbaaacbcbcbaabcbbbbbccbaaababcabababbbbbacbbaabcabaaccbcababbcacacaaabbabcbaacaaabaabbaccbaacccbcbbabbcacaababcbcaaaccccaabbccacbabbbacbbcbccbaacaabcbacbacbcaccbbccbbaabcabcbbcbbcbacbacccbcccbabcbbcacccabcbcbcababbaaaabbababbababaaabbbccaaacccaacaccbabaaabbbacccbbaabccbbcabacabbbbabccccaaaccccbccccbaaccbccbbcabbbcabaaaacbabaaabacbaabbbbcbaacabaccaabcaacbacccbabaccbbbbaabacbbabacacbaacabbbaacacccaaabccbcacccccaaabaccacbcbcbcabcababacbcccbcabbcaabcaabcaccacacccabcaabcbbcaccacbcacbaccbaabccababacaabaacbcacccacbcacabb

In [17]:
ord('a')

97